**Pre-signed URL**

_"Pre-signed URLs allow you to give your users access to a specific object
in your bucket without requiring them to have AWS security credentials or permissions."_

There are two types of pre-signed urls that can be generated:
1. GET pre-sign url (limited functionality)
2. POST pre-sign url (extended functionality)

In order to generate pre-signed URL, firstly import needed libraries:

In [ ]:
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import environ
import os

Function for pre-sign url with limited functionality:

In [ ]:
def create_get_presigned_url(bucket_name, object_name):
    """Generate a presigned URL S3 POST request to upload a file

    :param bucket_name: string
    :param object_name: string
    :return: Dictionary with the following keys:
        url: URL to post to
        fields: Dictionary of form fields and values to submit with the POST
    :return: None if error.
    """
    # Generate a pre-signed S3 POST URL
    s3_client = boto3.client('s3', config=Config(signature_version='s3v4'), region_name="eu-central-1")
    try:
        response = s3_client.generate_presigned_url('put_object',
                                                    Params={'Bucket': bucket_name,
                                                            'Key': object_name,
                                                            'ContentType': "application/json",
                                                            },
                                                    ExpiresIn=3600)
    except ClientError as e:
        print(str(e))
        return None

    # The response contains the pre-signed URL and required fields
    return response

Function for pre-sign url with extended functionality:

In [ ]:
def create_presigned_post(bucket_name, object_name, fields=None, conditions=None, expiration=3600):
    # Generate a presigned S3 POST URL
    s3_client = boto3.client('s3', config=Config(signature_version='s3v4'), region_name="eu-central-1")
    try:
        response = s3_client.generate_presigned_post(
            Bucket=bucket_name,
            Key=object_name,
            Fields=fields,
            Conditions=conditions,
            ExpiresIn=expiration
        )
    except ClientError as e:
        print(str(e))
        return None

    pre_signed_url = response['url']
    required_fields = response['fields']

    return pre_signed_url, required_fields

Set your env variables and specify file path. Then use generating function adjusted to your needs.

In [ ]:
if __name__ == '__main__':
    env = environ.Env()
    os.environ["AWS_ACCESS_KEY_ID"] = env("AWS_ACCESS_KEY_ID")
    os.environ["AWS_SECRET_ACCESS_KEY"] = env("AWS_SECRET_ACCESS_KEY")
    filename = "received/driving_session_id/trip_date/trip_id/rrrh-cs_1.parquet"
    bucket_name = env("BUCKET")

    # for extended functionality:
    # to add meta data to your file set fields and conditions with x-amz-meta-key: value

    fields = {"ACL": "bucket-owner-full-control", "x-amz-meta-random-meta": "test-value"}
    conditions = [{"ACL": "bucket-owner-full-control"}, {"x-amz-meta-random-meta": "test-value"}]
    url, required_fields = create_presigned_post(bucket_name, filename, fields=fields, conditions=conditions,
                                            expiration=3600)

    #for limited functionality uncomment:
    # url = create_get_presigned_url(bucket_name, filename)

    print(url)

Generated pre-signed url (and required_fields - in case of post pre-signed url)
can be used then as arguments when running algorithm.

Look at the basic_ex.py for instruction of how to initialize client and get algorithms
that client has permission to run.

In order to run algorithm with limited functionality pre-signed URL,
simply add generated url value to **callback_url** arg.

Example:

In [ ]:
results = algorithm.run(data, evaluate=False, encrypt=True, callback_url=generated_url)

In case of post pre-signed url with extended functionality, also
**callback_param** argument needs to be included when running algorithm.

Use required_fields from create_presigned_post() results in order to create
callback_param and assign it to "s3_fields" key.
Example:

In [ ]:
callback_param = {'s3_fields': required_fields}

# Now add callback_param argument to run() method. Obviously, generated url needs to be set as value of
# callback_url.
results = algorithm.run(data, evaluate=False, encrypt=True, callback_url=generated_url, callback_param=callback_param)
